In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import networkx as nx
from tqdm import tqdm
from numba import jit, cuda,vectorize
import scipy

In [2]:
scipy.__version__

'1.9.0'

In [3]:
train_graph=nx.read_edgelist('train_pos_after_eda.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
print(nx.info(train_graph))

C:\Users\Admin\AppData\Local\Temp\ipykernel_18384\1046942481.py:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(train_graph))


DiGraph with 1780768 nodes and 7550015 edges


## Similarity Measures

### Jaccard Distance:
##### The Jaccard similarity index (sometimes called the Jaccard similarity coefficient) compares members for two sets to see which members are shared and which are distinct. It’s a measure of similarity for the two sets of data, with a range from 0% to 100%. The higher the percentage, the more similar the two populations.

https://www.statisticshowto.com/jaccard-index/



In [4]:
def Jaccard_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a)))==0|len(set(train_graph.successors(b)))==b:  
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                (len(set(train_graph.successors(a)).union(set(train_graph.successors(b)))))
    except:
        return 0
    return sim

In [5]:
def Jaccard_for_followers(a,b):
    try:
        if len(set(train_graph.predecessors(a))) == 0  | len(set(g.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                 (len(set(train_graph.predecessors(a)).union(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

## Cosine distance
##### Cosine similarity is a metric, helpful in determining, how similar the data objects are irrespective of their size. We can measure the similarity between two sentences in Python using Cosine Similarity.
\begin{equation}
CosineDistance = \frac{|X\cap Y|}{|X|\cdot|Y|} 
\end{equation}

In [6]:
def Cosine_for_followees(a,b):
    try:
        if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
            return 0
        sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
                                    (math.sqrt(len(set(train_graph.successors(a)))*len((set(train_graph.successors(b))))))
        return sim
    except:
        return 0

In [7]:
def Cosine_for_followers(a,b):
    try:
        
        if len(set(train_graph.predecessors(a))) == 0  | len(set(train_graph.predecessors(b))) == 0:
            return 0
        sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
                                     (math.sqrt(len(set(train_graph.predecessors(a))))*(len(set(train_graph.predecessors(b)))))
        return sim
    except:
        return 0

## Page Ranking

##### PageRank (PR) is an algorithm used by Google Search to rank websites in their search engine results. PageRank was named after Larry Page, one of the founders of Google. PageRank is a way of measuring the importance of website pages.
##### PageRank is a link analysis algorithm and it assigns a numerical weighting to each element of a hyperlinked set of documents, such as the World Wide Web, with the purpose of "measuring" its relative importance within the set. The algorithm may be applied to any collection of entities with reciprocal quotations and references. 

https://en.wikipedia.org/wiki/PageRank#:~:text=PageRank%20(PR)%20is%20an%20algorithm,the%20importance%20of%20website%20pages.
https://www.geeksforgeeks.org/page-rank-algorithm-implementation/

In [8]:
Page_rank=nx.pagerank(train_graph)


In [9]:
mean_page_rank=sum(Page_rank.values()) / len(Page_rank)

### Shortest Path
##### Getting shortest path between two nodes, if nodes have direct path i.e directly connected then we are removing that edge and calculating path.

In [49]:
def Compute_shortest_path_length(a,b):
    p=-1
    try:
        if train_graph.has_edge(a,b):
            train_graph.remove_edge(a,b)
            p=nx.shortest_path_length(train_graph,source=a,target=b)
            train_graph.add_edge(a,b)
        else:
            p=nx.shortest_path_length(train_graph,source=a,target=b)
        return p
    except:
        return -1
    

In [50]:
Compute_shortest_path_length(1, 12)

8

### Checking for same community

In [11]:
wcc=list(nx.weakly_connected_components(train_graph))
def Belongs_to_same_wcc(a,b):
    index=[]
    
    if train_graph.has_edge(b,a):
        return 1
    if train_graph.has_edge(a,b):
        for i in wcc:
            if a in i:
                index=i
                break
        if b in index:
            train_graph.remove_edge(a,b)
            if Compute_shortest_path_length(a,b)==1:
                train_graph.add_edge(a,b)
                return 0
            else:
                train_graph.add_edge(a,b)
                return 1
        else:
            return 0
    else:
        for i in wcc:
            if a in i:
                index=i
                break
        if (b in index):
            return 1
        else:
            return 0

In [12]:
Belongs_to_same_wcc(861, 1659750)


1

In [13]:
Belongs_to_same_wcc(669354,1635354)

0

## Adamic /Adar Index:
#### Adamic/Adar measures is defined as inverted sum of degrees of common neighbours for given two vertices.
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

https://en.wikipedia.org/wiki/Adamic%E2%80%93Adar_index

In [14]:
def calc_adar_in(a,b):
    sum=0
    try:
        n=list(set(train_graph.successors(a)).intersection(set(train_graph.successors(b))))
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(train_graph.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0

In [15]:
calc_adar_in(1635354,669354)

0

## Is person was following back

In [16]:
def Follows_back(a,b):
    if train_graph.has_edge(b,a):
        return 1
    else:
        return 0

In [17]:
Follows_back(1,189226)

1

## Katz Centrality:
https://en.wikipedia.org/wiki/Katz_centrality

https://www.geeksforgeeks.org/katz-centrality-centrality-measure/

#### In graph theory, the Katz centrality of a node is a measure of centrality in a network. It was introduced by Leo Katz in 1953 and is used to measure the relative degree of influence of an actor (or node) within a social network. Unlike typical centrality measures which consider only the shortest path (the geodesic) between a pair of actors, Katz centrality measures influence by taking into account the total number of walks between a pair of actors. 
 
 $$x_i = \alpha \sum_{j} A_{ij} x_j + \beta,$$
where `A` is the adjacency matrix of the graph G 
with eigenvalues $$\lambda$$.



#### The parameter $$\beta$$ 
##### controls the initial centrality and 

$$\alpha < \frac{1}{\lambda_{max}}.$$


In [18]:
Katz=nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
pickle.dump(Katz,open('katz.p','wb'))

In [19]:
print('min',Katz[min(Katz, key=Katz.get)])
print('max',Katz[max(Katz, key=Katz.get)])
print('mean',float(sum(Katz.values())) / len(Katz))
mean_katz=float(sum(Katz.values())) / len(Katz)
print(Katz[20])

min 0.0007313499427279523
max 0.003391026125720957
mean 0.0007483743015672631
0.0007313499427279523


## Hits Score

##### Hyperlink Induced Topic Search (HITS) Algorithm is a Link Analysis Algorithm that rates webpages, developed by Jon Kleinberg. This algorithm is used to the web link-structures to discover and rank the webpages relevant for a particular search. HITS uses hubs and authorities to define a recursive relationship between webpages. Before understanding the HITS Algorithm, we first need to know about Hubs and Authorities.

https://www.geeksforgeeks.org/hyperlink-induced-topic-search-hits-algorithm-using-networxx-module-python/

https://en.wikipedia.org/wiki/HITS_algorithm

In [20]:
Hits = nx.hits(train_graph, max_iter=100, tol=1e-08, nstart=None, normalized=True)
pickle.dump(Hits,open('hits.p','wb'))

C:\Users\Admin\anaconda3\lib\site-packages\networkx\algorithms\link_analysis\hits_alg.py:78: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G, nodelist=list(G), dtype=float)


In [21]:
print('min',Hits[0][min(Hits[0], key=Hits[0].get)])
print('max',Hits[0][max(Hits[0], key=Hits[0].get)])
print('mean',float(sum(Hits[0].values())) / len(Hits[0]))
mean_hits=float(sum(Hits[0].values())) / len(Hits[0])

print(Hits[0].get(1000))

min -1.894105040621504e-20
max 0.004768353601532135
mean 5.615554637072843e-07
1.4528404834713936e-16


## Featurization



In [22]:
df_train=pd.read_csv('train_after_eda.csv',names=['source_node', 'destination_node'])
df_train['indicator_link']=pd.read_csv('train_y.csv',names=['indicator_link'])

df_train.head()


,source_node,destination_node,indicator_link
0,1443525,1177229,1
1,582845,818904,1
2,764111,1729045,1
3,499479,828225,1
4,182676,1647409,1


In [23]:
df_test=pd.read_csv('test_after_eda.csv',names=['source_node','destination_node'])
df_test['indicator_link']=pd.read_csv('test_y.csv',names=['indicator_link'])

In [24]:
df_test.head()

,source_node,destination_node,indicator_link
0,925781,744483,1
1,1047516,1767158,1
2,1123446,798286,1
3,1093801,936448,1
4,1667658,649427,1


##### num followers_s : number of followers source node have 
##### num followees_s: number of followees source node have
##### num followers_d: number of followers destination node have
##### num followees_d: number of followees destination node have

###### inter_followers: intersection of followers between source and destination
###### inter_followees: intersection of followees between source and destination

In [25]:

def compute_features_stage1(df_final):
    num_followers_s=[]
    num_followees_s=[]
    num_followers_d=[]
    num_followees_d=[]
    inter_followers=[]
    
    inter_followees=[]
    for i,row in tqdm(df_final.iterrows()):
        try:
            s1=set(train_graph.predecessors(row['source_node']))
            s2=set(train_graph.successors(row['source_node']))
        except:
            s1=set()
            s2=set()
        try:
            d1=set(train_graph.predecessors(row['desination_node']))
            d2=set(train_graph.successors(row['destiantion_node']))
        except:
            d1=set()
            d2=set()
        num_followers_s.append(len(s1))
        num_followees_s.append(len(s2))
        num_followers_d.append(len(d1))
        num_followees_d.append(len(d2))
        inter_followers.append(len(s1.intersection(d1)))
        inter_followees.append(len(s2.intersection(d2)))
    return  num_followers_s,num_followees_s,num_followers_d,num_followees_d,inter_followers,inter_followees

In [26]:
    df_train['num_followers_s'], df_train['num_followers_d'], \
    df_train['num_followees_s'], df_train['num_followees_d'], \
    df_train['inter_followers'], df_train['inter_followees']= compute_features_stage1(df_train)
    df_test['num_followers_s'], df_test['num_followers_d'], \
    df_test['num_followees_s'], df_test['num_followees_d'], \
    df_test['inter_followers'], df_test['inter_followees']= compute_features_stage1(df_test)
    

15099763it [12:11, 20652.73it/s]
3774942it [02:57, 21242.37it/s]


In [27]:
df_test.head()


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees
0,925781,744483,1,8,8,0,0,0,0
1,1047516,1767158,1,23,18,0,0,0,0
2,1123446,798286,1,11,15,0,0,0,0
3,1093801,936448,1,2,2,0,0,0,0
4,1667658,649427,1,48,19,0,0,0,0


In [28]:
df_train.head()

,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees
0,1443525,1177229,1,37,11,0,0,0,0
1,582845,818904,1,4,2,0,0,0,0
2,764111,1729045,1,20,23,0,0,0,0
3,499479,828225,1,0,2,0,0,0,0
4,182676,1647409,1,10,22,0,0,0,0


### Adding all the features to the dataframe

In [29]:
def compute_features_stage2(dft):

    page_rank_s=[]
    page_rank_d=[]
   
    katz_s=[]
    katz_d=[]
    hits_s=[]
    hits_d=[]
    
    for i,row in tqdm(dft.iterrows()):
       
        page_rank_s.append(Page_rank.get(row['source_node'],mean_page_rank))
        page_rank_d.append(Page_rank.get(row['destination_node'],mean_page_rank))
        katz_s.append(Katz.get(row['source_node'],mean_katz))
        katz_d.append(Katz.get(row['destination_node'],mean_katz))
        hits_s.append(Hits[0].get(row['source_node'],mean_hits))
        hits_d.append(Hits[0].get(row['destination_node'],mean_hits))
        
   
       
    return page_rank_s,page_rank_d,katz_s,katz_d,hits_s,hits_d

In [30]:

df_train['page_rank_s'],df_train['page_rank_d'],\
df_train['katz_s'],df_train['katz_d'],df_train['hits_s'],df_train['hits_d']=compute_features_stage2(df_train)

15099763it [12:55, 19476.05it/s]


In [31]:
print(df_train.shape)
df_train.head()


(15099763, 15)


,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,page_rank_s,page_rank_d,katz_s,katz_d,hits_s,hits_d
0,1443525,1177229,1,37,11,0,0,0,0,1.850118e-06,1.756390e-06,0.000883,0.000874,9.536818e-13,4.565034e-11
1,582845,818904,1,4,2,0,0,0,0,7.622485e-07,1.222512e-06,0.000746,0.000785,2.834183e-17,9.197072e-14
2,764111,1729045,1,20,23,0,0,0,0,2.882923e-06,8.497469e-07,0.000810,0.000800,1.558661e-13,1.139772e-14
3,499479,828225,1,0,2,0,0,0,0,1.655959e-07,6.110965e-07,0.000731,0.000743,3.075192e-17,1.289906e-15
4,182676,1647409,1,10,22,0,0,0,0,4.898329e-07,3.031131e-07,0.000770,0.000751,5.601692e-09,1.093959e-11


In [32]:

df_test['page_rank_s'],df_test['page_rank_d'],\
df_test['katz_s'],df_test['katz_d'],df_test['hits_s'],df_test['hits_d']=compute_features_stage2(df_test)

3774942it [03:13, 19535.41it/s]


In [33]:
def compute_features_stage3(dft):
     
     same_coummunity=[]
     Adar_index=[]
     follows_back=[]
     jaccard_for_followees=[]
     jaccard_for_followers=[]
     cosine_for_followees=[]
     cosine_for_followers=[]
     for i,row in tqdm(dft.iterrows()):
             jaccard_for_followees.append(Jaccard_for_followees(row['source_node'],row['destination_node']))
             jaccard_for_followers.append(Jaccard_for_followers(row['source_node'],row['destination_node']))
             cosine_for_followees.append(Cosine_for_followees(row['source_node'],row['destination_node']))
             cosine_for_followers.append(Cosine_for_followers(row['source_node'],row['destination_node']))
              
        
             shortest_path.append(Compute_shortest_path_length(row['source_node'],row['destination_node']))
             same_coummunity.append(Belongs_to_same_wcc(row['source_node'],row['destination_node'])) 
             Adar_index.append(calc_adar_in(row['source_node'],row['destination_node']))
             follows_back.append(Follows_back(row['source_node'],row['destination_node']))
     return jaccard_for_followees,jaccard_for_followers,\
        cosine_for_followees,cosine_for_followers,\
        same_coummunity,Adar_index,follows_back

In [34]:
df_train['jaccard_for_followees'],df_train['jaccard_for_followers'],\
df_train['cosine_for_followees'],df_train['cosine_for_followers'],\
df_train['same_coummunity'],df_train['Adar_index'],df_train['follows_back'],\
=compute_features_stage3(df_train)

15099763it [4:05:45, 1024.02it/s]


In [35]:
df_test['jaccard_for_followees'],df_test['jaccard_for_followers'],\
df_test['cosine_for_followees'],df_test['cosine_for_followers'],\
df_test['same_coummunity'],df_test['Adar_index'],df_test['follows_back'],\
=compute_features_stage3(df_test)

3774942it [1:09:24, 906.43it/s]


In [36]:
df_train.head()

,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,page_rank_s,...,hits_s,hits_d,jaccard_for_followees,jaccard_for_followers,cosine_for_followees,cosine_for_followers,same_coummunity,Adar_index,follows_back,shortest_path
0,1443525,1177229,1,37,11,0,0,0,0,1.850118e-06,...,9.536818e-13,4.565034e-11,0.019608,0,0,0,1,0.480126,1,-1
1,582845,818904,1,4,2,0,0,0,0,7.622485e-07,...,2.834183e-17,9.197072e-14,0.000000,0,0,0,1,0.000000,1,-1
2,764111,1729045,1,20,23,0,0,0,0,2.882923e-06,...,1.558661e-13,1.139772e-14,0.000000,0,0,0,1,0.000000,1,-1
3,499479,828225,1,0,2,0,0,0,0,1.655959e-07,...,3.075192e-17,1.289906e-15,0.000000,0,0,0,1,0.000000,0,-1
4,182676,1647409,1,10,22,0,0,0,0,4.898329e-07,...,5.601692e-09,1.093959e-11,0.535714,0,0,0,1,15.468246,1,-1


In [59]:
def Compute_features_stage4(dft):
    shortest_path=[]
    for i,row in tqdm(dft.iterrows()):
        shortest_path.append(Compute_shortest_path_length(row['source_node'],row['destination_node']))
        
    return shortest_path
        
    
    

In [56]:
df_train=df_train.drop(['shortest_path'], axis=1)
df_test=df_test.drop(['shortest_path'], axis=1)

In [60]:
df_train['shortest_path']=Compute_features_stage4(df_train)
df_test['shortest_path']=Compute_features_stage4(df_test)

15099763it [1:35:36, 2632.09it/s]
3774942it [23:24, 2688.10it/s]


In [61]:
df_test.head()

,source_node,destination_node,indicator_link,num_followers_s,num_followers_d,num_followees_s,num_followees_d,inter_followers,inter_followees,page_rank_s,...,hits_s,hits_d,jaccard_for_followees,jaccard_for_followers,cosine_for_followees,cosine_for_followers,same_coummunity,Adar_index,follows_back,shortest_path
0,925781,744483,1,8,8,0,0,0,0,1.233243e-06,...,8.102679e-17,4.691355e-18,0.000000,0,0,0,1,0.000000,1,-1
1,1047516,1767158,1,23,18,0,0,0,0,1.393932e-06,...,2.676223e-15,2.948024e-15,0.028571,0,0,0,1,1.285097,1,2
2,1123446,798286,1,11,15,0,0,0,0,1.119169e-06,...,8.410698e-16,1.670059e-15,0.000000,0,0,0,1,0.000000,1,-1
3,1093801,936448,1,2,2,0,0,0,0,6.863109e-07,...,7.586266e-18,4.112738e-21,0.000000,0,0,0,1,0.000000,1,-1
4,1667658,649427,1,48,19,0,0,0,0,9.345963e-07,...,1.078604e-18,4.692323e-19,0.192308,0,0,0,1,3.111279,0,2


In [62]:
df_train.to_csv('FB_train_featurization.csv')
df_test.to_csv('FB_test_featurization.csv')

